### In this notebook we go over a few tips on how to make your julia code faster, and show some tools that can help in this process
Whenever thinking of writing fast code, understanding three components are often helpful
    1. How types work in Julia
    2. Memory management
    3. Tools available to study the code

## 1. Types and Type Stability
A function is type stable when you can derive what the output of the function needs to be. The example below will make it clearer. As a rule of thumb, type stable functions are faster.

In [1]:
function square_plus_one(v::T) where T <:Number
    g = v*v
    return g+1
end

square_plus_one (generic function with 1 method)

In [2]:
v = rand()

0.12569648996264005

In [3]:
@code_warntype square_plus_one(v)

Body::Float64
│╻  *2 1 ─ %1 = (Base.mul_float)(v, v)::Float64
│╻╷ +3 │   %2 = (Base.add_float)(%1, 1.0)::Float64
│    └──      return %2


In [4]:
w = 5

5

In [5]:
@code_warntype square_plus_one(w)

Body::Int64
│╻ *2 1 ─ %1 = (Base.mul_int)(v, v)::Int64
│╻ +3 │   %2 = (Base.add_int)(%1, 1)::Int64
│   └──      return %2


Great! In the above two examples, we were able to predict what the output will be. This is because:
```
function square_plus_one(v::T) where T <:Number
    g = v*v         # Type(T * T) ==> T
    return g+1      # Type(T + Int)) ==> "max" (T,Int)
end

```
Note that in both calls the return type was different, once `Float64` and once `Int64`. But the function is still type stable.

Now let's move to something more interesting. Let's create our first type.

In [6]:
mutable struct Cube
    length
    width
    height
end

In [7]:
volume(c::Cube) = c.length*c.width*c.height

volume (generic function with 1 method)

In [8]:
mutable struct Cube_typed
    length::Float64
    width::Float64
    height::Float64
end
volume(c::Cube_typed) = c.length*c.width*c.height

volume (generic function with 2 methods)

In [9]:
mutable struct Cube_parametric_typed{T <: Real}
    length::T
    width::T
    height::T
end
volume(c::Cube_parametric_typed) = c.length*c.width*c.height

volume (generic function with 3 methods)

In [10]:
c1 = Cube(1.1,1.2,1.3)
c2 = Cube_typed(1.1,1.2,1.3)
c3 = Cube_parametric_typed(1.1,1.2,1.3)
@show volume(c1) == volume(c2) == volume(c3)

volume(c1) == volume(c2) == volume(c3) = true


true

In [11]:
using BenchmarkTools
@btime volume(c1) # not typed
@btime volume(c2) # typed float
@btime volume(c3) # typed parametric

  38.933 ns (1 allocation: 16 bytes)
  11.219 ns (1 allocation: 16 bytes)
  30.495 ns (1 allocation: 16 bytes)


1.7160000000000002

The second and the third function calls are faster! Let's call `@code_warntype` and check type stability

In [12]:
@code_warntype volume(c1)

Body::Any
│╻ getproperty1 1 ─ %1 = (Base.getfield)(c, :length)::Any
││  │   %2 = (Base.getfield)(c, :width)::Any
││  │   %3 = (Base.getfield)(c, :height)::Any
│   │   %4 = (%1 * %2 * %3)::Any
│   └──      return %4


In [13]:
@code_warntype volume(c2)

Body::Float64
│╻  getproperty6 1 ─ %1 = (Base.getfield)(c, :length)::Float64
││   │   %2 = (Base.getfield)(c, :width)::Float64
││   │   %3 = (Base.getfield)(c, :height)::Float64
││╻  *  │   %4 = (Base.mul_float)(%1, %2)::Float64
│││  │   %5 = (Base.mul_float)(%4, %3)::Float64
│    └──      return %5


In [14]:
@code_warntype volume(c3)

Body::Float64
│╻  getproperty6 1 ─ %1 = (Base.getfield)(c, :length)::Float64
││   │   %2 = (Base.getfield)(c, :width)::Float64
││   │   %3 = (Base.getfield)(c, :height)::Float64
││╻  *  │   %4 = (Base.mul_float)(%1, %2)::Float64
│││  │   %5 = (Base.mul_float)(%4, %3)::Float64
│    └──      return %5


**Conclusion**: Types matter, when you know anything about the types of your variables, include them in your code to make it run faster

In [15]:
function zero_or_val(x::Real)
    if x >= 0
        return x
    else
        return 0
    end
end
@code_warntype zero_or_val(0.2)

Body::Union{Float64, Int64}
│╻╷  >=2 1 ─ %1 = π (0.0, Float64)
││┃│  <=  │   %2 = (Base.lt_float)(%1, x)::Bool
│││   │   %3 = π (0.0, Float64)
│││╻   ==  │   %4 = (Base.eq_float)(%3, x)::Bool
│││╻   &  │   %5 = (Base.and_int)(%4, true)::Bool
│││╻   |  │   %6 = (Base.or_int)(%2, %5)::Bool
│     └──      goto #3 if not %6
│   3 2 ─      return x
│   5 3 ─      return 0


In [16]:
function zero_or_val_stable(x::Real)
    if x >= 0
        y = x
    else
        y = 0
    end
    T = promote_type(typeof(x),Int)
    return T(y)
end
@code_warntype zero_or_val_stable(0.2)

Body::Float64
│╻╷  >=2 1 ─ %1  = π (0.0, Float64)
││┃│  <=  │   %2  = (Base.lt_float)(%1, x)::Bool
│││   │   %3  = π (0.0, Float64)
│││╻   ==  │   %4  = (Base.eq_float)(%3, x)::Bool
│││╻   &  │   %5  = (Base.and_int)(%4, true)::Bool
│││╻   |  │   %6  = (Base.or_int)(%2, %5)::Bool
│     └──       goto #3 if not %6
│   3 2 ─       goto #4
│     3 ─       nothing
│   7 4 ┄ %10 = φ (#2 => _2, #3 => 0)::Union{Float64, Int64}
│   8 │   %11 = (isa)(%10, Int64)::Bool
│     └──       goto #6 if not %11
│     5 ─ %13 = π (%10, Int64)
│╻   Type  │   %14 = (Base.sitofp)(Float64, %13)::Float64
│     └──       goto #9
│     6 ─ %16 = (isa)(%10, Float64)::Bool
│     └──       goto #8 if not %16
│     7 ─ %18 = π (%10, Float64)
│     └──       goto #9
│     8 ─       (Core.throw)(ErrorException("fatal error in type inference (type bound)"))
│     └──       $(Expr(:unreachable))
│     9 ┄ %22 = φ (#5 => %14, #7 => %18)::Float64
│     └──       return %22


**Conclusion**: You can avoid type instable code by using the `promote_type` function which returns the highest of the two types passed.

Let us say we want to play the following game, I give you a vector of numbers. And you want to accumulate the sum as follows. For each number in the vector, you toss a coin (`rand()`), if it is heads (`>=0.5`), you add `1`. Otherwise, you add the number itself.

In [17]:
function flipcoin_then_add(v::Vector{T}) where T <: Real
    s = 0
    for vi in v
        r = rand()
        if r >=0.5
            s += 1
        else
            s += vi
        end
    end
end

function flipcoin_then_add_typed(v::Vector{T}) where T <: Real
    s = zero(T)
    for vi in v
        r = rand()
        if r >=0.5
            s += one(T)
        else
            s += vi
        end
    end
end
myvec = rand(1000)
@show flipcoin_then_add(myvec) == flipcoin_then_add_typed(myvec)

flipcoin_then_add(myvec) == flipcoin_then_add_typed(myvec) = true


true

In [18]:
@btime flipcoin_then_add(rand(1000))
@btime flipcoin_then_add_typed(rand(1000))

  10.286 μs (1 allocation: 7.94 KiB)
  3.425 μs (1 allocation: 7.94 KiB)


**Conclusion**: Think about the variables you are declaring. Do you know their types? If so, specify the type somehow.

## 2. Memory
As a rule of thumb, **functions with preallocated memory run faster**

A classic example here is to build an array with pre-allocated memory versus pushing to it. Let's try to build Fibonacci using both approaches

In [19]:
function build_fibonacci_preallocate(n::Int)
    @assert n >= 2
    v = zeros(Int64,n)
    v[1] = 1
    v[2] = 1
    for i = 3:n
        v[i] = v[i-1] + v[i-2]
    end
    return v
end

build_fibonacci_preallocate (generic function with 1 method)

In [20]:
function build_fibonacci_no_allocation(n::Int)
    @assert n >= 2
    v = Vector{Int64}()
    push!(v,1)
    push!(v,1)
    for i = 3:n
        push!(v,v[i-1]+v[i-2])
    end
    return v
end

build_fibonacci_no_allocation (generic function with 1 method)

In [21]:
@show isequal(build_fibonacci_preallocate(10),build_fibonacci_no_allocation(10))

isequal(build_fibonacci_preallocate(10), build_fibonacci_no_allocation(10)) = true


true

In [22]:
n = 100
@btime build_fibonacci_no_allocation(n);
@btime build_fibonacci_preallocate(n);

  1.265 μs (7 allocations: 2.27 KiB)
  255.648 ns (1 allocation: 896 bytes)


**Conclusion**: Whenever possible, preallocate memory

It's also important to understand **how memory is organized in Julia**. Let's say, for _some reason_ you want to access all the elements of a matrix once. For the sake of this experiment, let's say we want to write `matrix_sum(A)` where A is a matrix

In [23]:
# Create a random matrix A of size m-by-n
m = 10000
n = 10000
A = rand(m,n)
;

In [24]:
function matrix_sum_rows(A::Matrix)
    m,n = size(A)
    mysum = 0
    for i = 1:m # fix a row
        for j = 1:n # loop over cols
            mysum += A[i,j]
        end
    end
    return mysum
end

matrix_sum_rows (generic function with 1 method)

In [25]:
function matrix_sum_cols(A::Matrix)
    m,n = size(A)
    mysum = 0
    for j = 1:n # fix a column
        for i = 1:m # loop over rows
            mysum += A[i,j]
        end
    end
    return mysum
end

matrix_sum_cols (generic function with 1 method)

In [26]:
function matrix_sum_index(A::Matrix)
    m,n = size(A)
    mysum = 0
    for i = 1:m*n
        mysum += A[i]
    end
    return mysum
end

matrix_sum_index (generic function with 1 method)

In [27]:
@show matrix_sum_cols(A) ≈ matrix_sum_rows(A) ≈ matrix_sum_index(A)

matrix_sum_cols(A) ≈ matrix_sum_rows(A) ≈ matrix_sum_index(A) = true


true

In [28]:
@btime matrix_sum_rows(A)
@btime matrix_sum_cols(A)
@btime matrix_sum_index(A)

  2.247 s (1 allocation: 16 bytes)
  122.958 ms (1 allocation: 16 bytes)
  131.959 ms (1 allocation: 16 bytes)


5.000175594658105e7

**Conclusion**: Matrices are orgnaized column-wise in memory. It's better to access them one column at a time. Consider understanding how your data is organized in memory when you want to access it.

**Memory recyling** is also an important concept that can make your code run faster. **Bonus in this example**: vectorized operations are not necessairly faster.

Let's take this example, you have a vector b and a vector h where b[i] is the base length of triangle i and h[i] is the height length. The experiment is to find the hypotenuse value of all triangles

In [29]:
b = rand(1000)*10
h = rand(1000)*10
function find_hypotenuse(b::Vector{T},h::Vector{T}) where T <: Real
    return sqrt.(b.^2+h.^2)
end

find_hypotenuse (generic function with 1 method)

In [30]:
# Let's time it
@btime find_hypotenuse(b,h);

  10.556 μs (6 allocations: 31.81 KiB)


In [31]:
function find_hypotenuse_optimized(b::Vector{T},h::Vector{T}) where T <: Real
    accum_vec = similar(b)
    for i = 1:length(b)
        accum_vec[i] = b[i]^2
        accum_vec[i] = accum_vec[i] + h[i]^2 # here, we used the same space in memory to hold the sum
        accum_vec[i] = sqrt(accum_vec[i]) # same thing here, to hold the sqrt
        # or:
        # accum_vec[i] = sqrt(b[i]^2+h[i]^2)
    end
    return accum_vec
end
@btime find_hypotenuse_optimized(b,h);

  7.175 μs (1 allocation: 7.94 KiB)


**Conclusion**: Whenever you can reuse memory, reuse it. 

## 3. Tools
It is helpful to learn about some of the tools that Julia provides to make your code faster

### @inbounds

In [32]:
?@inbounds

```
@inbounds(blk)
```

Eliminates array bounds checking within expressions.

In the example below the in-range check for referencing element `i` of array `A` is skipped to improve performance.

```julia
function sum(A::AbstractArray)
    r = zero(eltype(A))
    for i = 1:length(A)
        @inbounds r += A[i]
    end
    return r
end
```

!!! warning
    Using `@inbounds` may return incorrect results/crashes/corruption for out-of-bounds indices. The user is responsible for checking it manually. Only use `@inbounds` when it is certain from the information locally available that all accesses are in bounds.



In [33]:
# Let us say we want to find the sum of all elements in a vector

In [34]:
function new_sum(myvec::Vector{Int})
    s = 0
    for i = 1:length(myvec)
        s += myvec[i]
    end
    return s
end

function new_sum_inbounds(myvec::Vector{Int})
    s = 0
    @inbounds for i = 1:length(myvec)
        s += myvec[i]
    end
    return s
end

new_sum_inbounds (generic function with 1 method)

In [35]:
myvec = collect(1:1000000)
@btime new_sum(myvec)
@btime new_sum_inbounds(myvec)

  841.042 μs (0 allocations: 0 bytes)
  511.709 μs (0 allocations: 0 bytes)


500000500000

In [36]:
@show isequal(new_sum(myvec),new_sum_inbounds(myvec))

isequal(new_sum(myvec), new_sum_inbounds(myvec)) = true


true

In [37]:
# Be careful though!
function new_sum_WRONG(myvec::Vector{Int})
    s = 0
    for i = 1:length(myvec)+1
        s += myvec[i]
    end
    return s
end

function new_sum_inbounds_WRONG(myvec::Vector{Int})
    s = 0
    @inbounds for i = 1:length(myvec)+1
        s += myvec[i]
    end
    return s
end

myvec = collect(1:1000000);

In [38]:
@btime new_sum_WRONG(myvec)

BoundsError: BoundsError: attempt to access 1000000-element Array{Int64,1} at index [1000001]

In [39]:
@btime new_sum_inbounds_WRONG(myvec) # this actually exectued!

  511.546 μs (0 allocations: 0 bytes)


500000500000

### @code_XXX
One cool thing about Julia is that it allows you to see the different stages of the code before all the way to native code! Let's look at these macros that allow you to achieve that

In [40]:
# @code_llvm 
# @code_lowered 
# @code_native 
# @code_typed 
# @code_warntype

function flipcoin(randval::Float64)
    if randval<0.5
        return "H"
    else
        return "T"
    end
end

flipcoin (generic function with 1 method)

In [41]:
@code_lowered flipcoin(rand()) # syntax tree

CodeInfo(
│8  1 ─ %1 = randval < 0.5
│   └──      goto #3 if not %1
│9  2 ─      return "H"
│11 3 ─      return "T"
)

In [42]:
@code_warntype flipcoin(rand()) # try @code_typed

Body::String
│╻ <8  1 ─ %1 = (Base.lt_float)(randval, 0.5)::Bool
│    └──      goto #3 if not %1
│ 9  2 ─      return "H"
│ 11 3 ─      return "T"


In [43]:
@code_llvm flipcoin(rand()) # this and code_warntype are probably the most relevant


; Function flipcoin
; Location: In[40]:8
define nonnull %jl_value_t addrspace(10)* @julia_flipcoin_37161(double) {
top:
; Function <; {
; Location: float.jl:452
  %1 = fcmp uge double %0, 5.000000e-01
;}
  %spec.select = select i1 %1, %jl_value_t addrspace(10)* addrspacecast (%jl_value_t* inttoptr (i64 4607296016 to %jl_value_t*) to %jl_value_t addrspace(10)*), %jl_value_t addrspace(10)* addrspacecast (%jl_value_t* inttoptr (i64 4607295888 to %jl_value_t*) to %jl_value_t addrspace(10)*)
; Location: In[40]:9
  ret %jl_value_t addrspace(10)* %spec.select
}


In [44]:
@code_native flipcoin(rand())

	.section	__TEXT,__text,regular,pure_instructions
; Function flipcoin {
; Location: In[40]:8
	decl	%eax
	movl	$1383627152, %eax       ## imm = 0x52787990
	addl	%eax, (%eax)
	addb	%al, (%eax)
; Function <; {
; Location: float.jl:452
	vmovsd	(%eax), %xmm1           ## xmm1 = mem[0],zero
	xorl	%ecx, %ecx
	vucomisd	%xmm0, %xmm1
	setbe	%cl
;}
	decl	%eax
	shll	$7, %ecx
	decl	%eax
	movl	$312328592, %eax        ## imm = 0x129DC190
	addl	%eax, (%eax)
	addb	%al, (%eax)
	decl	%eax
	addl	%ecx, %eax
; Location: In[40]:9
	retl
	nopl	(%eax)
;}
